### Lets start by importing the necassary packages and creating a connection to our database.

In [2]:
import sqlite3

In [3]:
conn = sqlite3.connect("fire.sqlite")

In [4]:
c = conn.cursor()

Choose not to pursue [normalization](https://www.techopedia.com/definition/1221/normalization) because the database is not big so join time will outweigh pros of reducing space.

### Next we grab our table names.

In [7]:
c.execute("SELECT name FROM sqlite_master WHERE type = 'table'")

This Points the cursor at what we want (name from the table sqlite_master)

In [ ]:
print c.fetchall()

This prints the values at the cursor.

### Next we look for the column names in our table.

In [9]:
c.execute("PRAGMA table_info(fire)")

In [10]:
print c.fetchall()

[(0, u'index', u'BIGINT', 0, None, 0), (1, u'Unnamed: 0', u'BIGINT', 0, None, 0), (2, u'FIRE_NAME', u'TEXT', 0, None, 0), (3, u'FIRE_DAY', u'BIGINT', 0, None, 0), (4, u'FIRE_MONTH', u'BIGINT', 0, None, 0), (5, u'FIRE_YEAR', u'BIGINT', 0, None, 0), (6, u'DATA_SOURC', u'TEXT', 0, None, 0), (7, u'CAUSE', u'BIGINT', 0, None, 0), (8, u'REPORTED_A', u'BIGINT', 0, None, 0), (9, u'SIZE_CLASS', u'TEXT', 0, None, 0), (10, u'AGENCY', u'TEXT', 0, None, 0), (11, u'COMMENTS', u'FLOAT', 0, None, 0), (12, u'IGNITION_D', u'BIGINT', 0, None, 0), (13, u'CONT_DATE', u'BIGINT', 0, None, 0), (14, u'REGION_NUM', u'BIGINT', 0, None, 0), (15, u'FOREST_NUM', u'BIGINT', 0, None, 0), (16, u'ICS_CODE', u'TEXT', 0, None, 0), (17, u'ACRES', u'FLOAT', 0, None, 0), (18, u'HA', u'FLOAT', 0, None, 0), (19, u'SIZE_CLASS_', u'TEXT', 0, None, 0)]


Note that this command gives lots of information, all we really need for now is Column Name.
The format goes (COLUMN NUMBER,u'COLUMN NAME',u'DATA TYPE',...)

### Next let's find only the fires caused by lightning
CAUSE is the 8th column (count starts at 0).
Note that the CAUSE code for lightning is 1, this is outside information not included in the table.

In [11]:
c.execute("SELECT * FROM fire WHERE CAUSE == 1")

In [12]:
print c.fetchall()

[(3, 4, u'TELLS CK', 23, 9, 1924, u'UNKNOWN', 1, 0, u'C', u'USF', None, 19240923, 19240000, 5, 3, u'ENF', 15.9, 6.44, u'10-99.9 acres'), (11, 12, None, 0, 0, 1926, u'UNKNOWN', 1, 0, u'D', u'USF', None, 19260000, 19260000, 5, 6, u'LNF', 214.63, 86.86, u'100-299.9 acres'), (19, 20, u'STICKER BASIN', 18, 6, 1949, u'UNKNOWN', 1, 0, u'C', u'USF', None, 19490618, 19490000, 5, 10, u'SRF', 34.22, 13.85, u'10-99.9 acres'), (26, 27, u'STATE LINE', 21, 7, 1945, u'UNKNOWN', 1, 0, u'C', u'USF', None, 19450721, 19450000, 5, 9, u'MDF', 68.59, 27.76, u'10-99.9 acres'), (30, 31, None, 0, 0, 1911, u'UNKNOWN', 1, 0, u'E', u'USF', None, 19110000, 19110000, 5, 15, u'SNF', 419.39, 169.72, u'300-999.9 acres'), (31, 32, None, 0, 0, 1917, u'UNKNOWN', 1, 0, u'E', u'USF', None, 19170000, 19170000, 5, 6, u'LNF', 614.3, 248.6, u'300-999.9 acres'), (33, 34, None, 0, 0, 1917, u'UNKNOWN', 1, 0, u'E', u'USF', None, 19170000, 19170000, 5, 6, u'LNF', 327.39, 132.49, u'300-999.9 acres'), (36, 37, None, 0, 0, 1917, u'UNKN

This output is hard to read but just take a cursory glance and make sure the 8th spot is 1.

### Now that we have this selection lets make a new table based on it.
To create a new table from a subset of this existing table we will use CREATE TABLE AS, this syntax for which is:
CREATE TABLE new_table AS SELECT expressions FROM existing_tables WHERE conditions

In [13]:
c.execute("CREATE TABLE lightfire AS SELECT * FROM fire WHERE CAUSE == 1")

The cursor is a temporary operation, you must commit your operation to actually write to the sqlite file!

In [15]:
conn.commit()

Let's make sure our new table is in the database.

In [16]:
c.execute("SELECT name FROM sqlite_master WHERE type = 'table'")

In [17]:
print c.fetchall()

[(u'fire',), (u'lightfire',)]


In [18]:
c.execute("SELECT * FROM lightfire")

In [19]:
print c.fetchall()

[(3, 4, u'TELLS CK', 23, 9, 1924, u'UNKNOWN', 1, 0, u'C', u'USF', None, 19240923, 19240000, 5, 3, u'ENF', 15.9, 6.44, u'10-99.9 acres'), (11, 12, None, 0, 0, 1926, u'UNKNOWN', 1, 0, u'D', u'USF', None, 19260000, 19260000, 5, 6, u'LNF', 214.63, 86.86, u'100-299.9 acres'), (19, 20, u'STICKER BASIN', 18, 6, 1949, u'UNKNOWN', 1, 0, u'C', u'USF', None, 19490618, 19490000, 5, 10, u'SRF', 34.22, 13.85, u'10-99.9 acres'), (26, 27, u'STATE LINE', 21, 7, 1945, u'UNKNOWN', 1, 0, u'C', u'USF', None, 19450721, 19450000, 5, 9, u'MDF', 68.59, 27.76, u'10-99.9 acres'), (30, 31, None, 0, 0, 1911, u'UNKNOWN', 1, 0, u'E', u'USF', None, 19110000, 19110000, 5, 15, u'SNF', 419.39, 169.72, u'300-999.9 acres'), (31, 32, None, 0, 0, 1917, u'UNKNOWN', 1, 0, u'E', u'USF', None, 19170000, 19170000, 5, 6, u'LNF', 614.3, 248.6, u'300-999.9 acres'), (33, 34, None, 0, 0, 1917, u'UNKNOWN', 1, 0, u'E', u'USF', None, 19170000, 19170000, 5, 6, u'LNF', 327.39, 132.49, u'300-999.9 acres'), (36, 37, None, 0, 0, 1917, u'UNKN

Everything Looks Good!

Some additional queries:
Find the average number of fires that occurs in all of the forests.

In [ ]:
c.execute('SELECT AVG(counts) FROM (SELECT COUNT(*) AS counts FROM fire GROUP BY(FOREST_NUM))')

Return a list of tuples that have (cause, forest_number) grouped by the date the fire started and ordered by the forest number.

In [ ]:
c.execute('SELECT DISTINCT(CAUSE),FOREST_NUM FROM fire GROUP BY(IGNITION_D) ORDER BY(FOREST_NUM)')

Find the total number of forests in this dataset.

In [ ]:
c.execute('SELECT COUNT(DISTINCT(FOREST_NUM)) FROM fire')

Find the number of different dates fires ignited.

In [ ]:
c.execute('SELECT COUNT(DISTINCT(IGNITION_D)) FROM fire')